In [4]:
import pandas as pd
import pandasql as ps
import itertools 

matches = pd.read_csv("untransformed_1.csv")

######################################
# ligue 1 - 1843
# BPL - 2411
# La Liga - 1869
# Serie A - 1854
# Bundesliga - 1845
######################################

q1 = """SELECT * FROM matches
        WHERE Season = 2016"""

q2 = """SELECT * FROM matches
        WHERE Season = 2017"""

q3 = """SELECT * FROM matches
        WHERE Season = 2018"""

q4 = """SELECT * FROM matches
        WHERE Season = 2019"""

q5 = """SELECT * FROM matches
        WHERE Season = 2020"""

q6 = """SELECT * FROM matches
        WHERE Season = 2021"""

matches_prep_stats = []


matches_prep_stats.append(ps.sqldf(q1))
matches_prep_stats.append(ps.sqldf(q2))
matches_prep_stats.append(ps.sqldf(q3))
matches_prep_stats.append(ps.sqldf(q4))
matches_prep_stats.append(ps.sqldf(q5))
matches_prep_stats.append(ps.sqldf(q6))

matches_stats = []

league_id = [1843, 2411, 1869, 1854, 1845]
q = """SELECT * FROM matches
        WHERE league_id = """
for i in range(len(matches_prep_stats)):
    ms = matches_prep_stats[i]
    for j in range(len(league_id)):
        matches_stats.append(ps.sqldf("""SELECT * FROM ms 
                                     WHERE league_id = """+str(league_id[j])))
headers = ['Unnamed: 0', 'MatchID', 'season', 'datetime', 'league_id', 'league',
       'HomeID', 'HomeTeam', 'Home_short', 'days_home', 'AwayID', 'AwayTeam',
       'Away_short', 'days_away', 'distance(km)', 'spi1', 'spi2',
       'proj_score1', 'proj_score2', 'importance1', 'importance2', 'xg1',
       'xg2', 'nsxg1', 'nsxg2', 'adj_score1', 'adj_score2', 'home_xG',
       'away_xG', 'home_goals', 'away_goals', 'Results']

# add_data = [2, 12989, 2019, '3/7/2020  6:00:00 PM', ]
matches_stats[15]=matches_stats[15][0:270]
# matches_stats[25]=matches_stats[25][0:100]
# matches_stats[26]=matches_stats[26][0:90]
# matches_stats[27]=matches_stats[27][0:90]
# matches_stats[28]=matches_stats[28][0:90]
matches_stats.pop(25)
matches_stats.pop(26)

for i in range(len(matches_stats)):    
    print(i,len(matches_stats[i]))

0 380
1 380
2 380
3 380
4 306
5 380
6 380
7 380
8 380
9 306
10 380
11 380
12 380
13 380
14 306
15 270
16 380
17 380
18 380
19 306
20 380
21 380
22 380
23 380
24 306
25 120
26 130
27 108


In [6]:
# Gets the goals scored agg arranged by teams and matchweek

def get_goals_scored(playing_stat):
    global val
    if(len(playing_stat)%10 == 0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9 ==0):
        val = int(len(playing_stat)/9)+1

    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTGS = playing_stat.iloc[i]['home_goals']
        ATGS = playing_stat.iloc[i]['away_goals']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

# Gets the goals conceded agg arranged by teams and matchweek
def get_goals_conceded(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['home_goals']
        HTGC = playing_stat.iloc[i]['away_goals']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

# Gets the goals scored agg arranged by teams and matchweek
def get_xG_scored(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1

    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTxGS = playing_stat.iloc[i]['home_xG']
        ATxGS = playing_stat.iloc[i]['away_xG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTxGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATxGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    xGScored = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    xGScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        xGScored[i] = xGScored[i] + xGScored[i-1]
    return xGScored

# Gets the goals conceded agg arranged by teams and matchweek
def get_xG_conceded(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1

    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATxGC = playing_stat.iloc[i]['home_xG']
        HTxGC = playing_stat.iloc[i]['away_xG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTxGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATxGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    xGConceded = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    xGConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        xGConceded[i] = xGConceded[i] + xGConceded[i-1]
    return xGConceded

# Gets the goals scored agg arranged by teams and matchweek
def get_nsxG_scored(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1
    else:
        val= 28
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        HTnsxGS = playing_stat.iloc[i]['nsxg1']
        ATnsxGS = playing_stat.iloc[i]['nsxg2']
        teams[playing_stat.iloc[i].HomeTeam].append(HTnsxGS)
        teams[playing_stat.iloc[i].AwayTeam].append(ATnsxGS)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    nsxGScored = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    nsxGScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        nsxGScored[i] = nsxGScored[i] + nsxGScored[i-1]
    return nsxGScored

# Gets the goals conceded agg arranged by teams and matchweek
def get_nsxG_conceded(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1

    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATnsxGC = playing_stat.iloc[i]['nsxg1']
        HTnsxGC = playing_stat.iloc[i]['nsxg2']
        teams[playing_stat.iloc[i].HomeTeam].append(HTnsxGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATnsxGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    nsxGConceded = pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T
    nsxGConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,val):
        nsxGConceded[i] = nsxGConceded[i] + nsxGConceded[i-1]
    return nsxGConceded

def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_scored(playing_stat)
    xGS = get_xG_scored(playing_stat)
    xGC = get_xG_conceded(playing_stat)
    nsxGS = get_nsxG_scored(playing_stat)
    nsxGC = get_nsxG_conceded(playing_stat)
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []
    HTxGS = []
    ATxGS = []
    HTxGC = []
    ATxGC = []
    HTnsxGS = []
    ATnsxGS = []
    HTnsxGC = []
    ATnsxGC = []

    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        HTxGS.append(xGS.loc[ht][j])
        ATxGS.append(xGS.loc[at][j])
        HTxGC.append(xGC.loc[ht][j])
        ATxGC.append(xGC.loc[at][j])
        HTnsxGS.append(nsxGS.loc[ht][j])
        ATnsxGS.append(nsxGS.loc[at][j])
        HTnsxGC.append(nsxGC.loc[ht][j])
        ATnsxGC.append(nsxGC.loc[at][j])
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    playing_stat['Home_goals_rolling'] = HTGS
    playing_stat['Away_goals_rolling'] = ATGS
    playing_stat['Home_conceded_rolling'] = HTGC
    playing_stat['Away_conceded_rolling'] = ATGC
    playing_stat['Home_xG_rolling'] = HTxGS
    playing_stat['Away_xG_rolling'] = ATxGS
    playing_stat['Home_xGA_rolling'] = HTxGC
    playing_stat['Away_xGA_rolling'] = ATxGC
    playing_stat['Home_nsxG_rolling'] = HTnsxGS
    playing_stat['Away_nsxG_rolling'] = ATnsxGS
    playing_stat['Home_nsxGA_rolling'] = HTnsxGC
    playing_stat['Away_nsxGA_rolling'] = ATnsxGC
    
    return playing_stat


for l in range(len(matches_stats)):
    matches_stats[l] = get_gss(matches_stats[l])
    
for i in range(len(matches_stats)):
    if(len(matches_stats[i])%10==0):
        for j in range(1,int(len(matches_stats[i])/10)):
            matches_stats[i]['Home_goals_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Home_goals_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_goals_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Away_goals_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Home_conceded_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Home_conceded_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_conceded_rolling'][10*j:10*(j+1)]=(matches_stats[i]['Away_conceded_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Home_xG_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Home_xG_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_xG_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Away_xG_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Home_xGA_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Home_xGA_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_xGA_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Away_xGA_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Home_nsxG_rolling'][10*j:10*(j+1)]=(matches_stats[i]['Home_nsxG_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_nsxG_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Away_nsxG_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Home_nsxGA_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Home_nsxGA_rolling'][10*j:10*(j+1)])/j
            matches_stats[i]['Away_nsxGA_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Away_nsxGA_rolling'][10*j:10*(j+1)])/j
    elif(len(matches_stats[i])%9==0):
        for j in range(1,int(len(matches_stats[i])/9)):
            matches_stats[i]['Home_goals_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_goals_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_goals_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_goals_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Home_conceded_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_conceded_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_conceded_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_conceded_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Home_xG_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_xG_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_xG_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_xG_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Home_xGA_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_xGA_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_xGA_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_xGA_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Home_nsxG_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_nsxG_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_nsxG_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_nsxG_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Home_nsxGA_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Home_nsxGA_rolling'][9*j:9*(j+1)])/j
            matches_stats[i]['Away_nsxGA_rolling'][9*j:9*(j+1)]=(matches_stats[i]['Away_nsxGA_rolling'][9*j:9*(j+1)])/j

            

<ipython-input-6-dc85a214811e>:231: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_stats[i]['Home_goals_rolling'][10*j:10*(j+1)] = (matches_stats[i]['Home_goals_rolling'][10*j:10*(j+1)])/j
<ipython-input-6-dc85a214811e>:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_stats[i]['Away_goals_rolling'][10*j:10*(j+1)]= (matches_stats[i]['Away_goals_rolling'][10*j:10*(j+1)])/j
<ipython-input-6-dc85a214811e>:233: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [7]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0
    

def get_cuml_points(matchres,playing_stat):
    global val
    global val1
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
        val1=20
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1
        val1=18
        
    matchres_points = matchres.applymap(get_points)
    for i in range(2,val):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]
        
    matchres_points.insert(column =0, loc = 0, value = [0*i for i in range(val1)])
    return matchres_points

def get_matchres(playing_stat):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].Results == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].Results == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,val)]).T

def get_agg_points(playing_stat):

    
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres,playing_stat)
    HTP = []
    ATP = []
    j = 0
    for i in range(len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

for e in range(len(matches_stats)):
    matches_stats[e] = get_agg_points(matches_stats[e])

In [8]:
def get_form(playing_stat,num):
    global val
    if(len(playing_stat)%10==0):
        val = int(len(playing_stat)/10)+1
    elif(len(playing_stat)%9==0):
        val = int(len(playing_stat)/9)+1
    form = get_matchres(playing_stat)
    form_final = form.copy()
    for i in range(num,val):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(playing_stat,num):
    form = get_form(playing_stat,num)
    h = ['M' for i in range(num * 10)]  # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),len(playing_stat)):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               # get past n results
        h.append(past[num-1])                    # 0 index is most recent
        
        past = form.loc[at][j]               # get past n results.
        a.append(past[num-1])                   # 0 index is most recent
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    playing_stat['HM' + str(num)] = h                 
    playing_stat['AM' + str(num)] = a

    
    return playing_stat

def add_form_df(playing_statistics):
    playing_statistics = add_form(playing_statistics,1)
    playing_statistics = add_form(playing_statistics,2)
    playing_statistics = add_form(playing_statistics,3)
    playing_statistics = add_form(playing_statistics,4)
    playing_statistics = add_form(playing_statistics,5)
    playing_statistics = add_form(playing_statistics,1)
    return playing_statistics



for s in range(len(matches_stats)):
    matches_stats[s] = add_form_df(matches_stats[s])


In [9]:
cols =         ['MatchID','HomeTeam','AwayTeam', 'spi1','spi2','importance1','importance2',
                'home_goals', 'away_goals', 'home_xG', 'away_xG','nsxg1','nsxg2',
                'days_home','days_away','Results','distance(km)',
               'Home_goals_rolling', 'Away_goals_rolling', 'Home_conceded_rolling', 'Away_conceded_rolling', 
                'Home_xG_rolling', 'Away_xG_rolling', 'Home_xGA_rolling', 'Away_xGA_rolling', 
                'Home_nsxG_rolling', 'Away_nsxG_rolling', 'Home_nsxGA_rolling', 'Away_nsxGA_rolling',
               'HTP', 'ATP', 'HM1', 'HM2', 'HM3','HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']



for r in range(len(matches_stats)):
    matches_stats[r] = matches_stats[r][cols]


def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(len(playing_stat)):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

for a in range(len(matches_stats)):
    matches_stats[a] = get_mw(matches_stats[a])

for r in range(len(matches_stats)):
    matches_stats[r] = matches_stats[r][51:]


matches_stat = pd.concat(matches_stats,ignore_index=True)


# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

matches_stat['HTFormPtsStr'] = matches_stat['HM1'] + matches_stat['HM2'] + matches_stat['HM3'] + matches_stat['HM4'] + matches_stat['HM5']
matches_stat['ATFormPtsStr'] = matches_stat['AM1'] + matches_stat['AM2'] + matches_stat['AM3'] + matches_stat['AM4'] + matches_stat['AM5']

matches_stat['HTFormPts'] = matches_stat['HTFormPtsStr'].apply(get_form_points)
matches_stat['ATFormPts'] = matches_stat['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0

matches_stat['HTWinStreak3'] = matches_stat['HTFormPtsStr'].apply(get_3game_ws)
matches_stat['HTWinStreak5'] = matches_stat['HTFormPtsStr'].apply(get_5game_ws)
matches_stat['HTLossStreak3'] = matches_stat['HTFormPtsStr'].apply(get_3game_ls)
matches_stat['HTLossStreak5'] = matches_stat['HTFormPtsStr'].apply(get_5game_ls)

matches_stat['ATWinStreak3'] = matches_stat['ATFormPtsStr'].apply(get_3game_ws)
matches_stat['ATWinStreak5'] = matches_stat['ATFormPtsStr'].apply(get_5game_ws)
matches_stat['ATLossStreak3'] = matches_stat['ATFormPtsStr'].apply(get_3game_ls)
matches_stat['ATLossStreak5'] = matches_stat['ATFormPtsStr'].apply(get_5game_ls)

matches_stat.keys()

matches_stat.to_csv("final_data1.csv")

<ipython-input-9-8883fba02aad>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  playing_stat['MW'] = MatchWeek
